<a href="https://colab.research.google.com/github/qilalifa/ImageDetection-Yolov8/blob/main/yolov8_custom_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# Pip install (recommended)

!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 695.2/695.2 kB 4.8 MB/s eta 0:00:00


# Creating train-val split

In [ ]:
## importing required libraries
import os
import shutil
import random

!pip install tqdm --upgrade
from tqdm.notebook import tqdm

In [ ]:
## connecting to the google drive

from google.colab import drive
drive.mount('/content/drive')


In [ ]:
train_path_img = "./yolo_data/images/train/"
train_path_label = "./yolo_data/labels/train/"
val_path_img = "./yolo_data/images/val/"
val_path_label = "./yolo_data/labels/val/"
test_path = "./yolo_data/test"

In [ ]:
'''
Split the dataset into train and test and creates the train.txt and test.tx with
the respective path of the images in each folder
'''

def train_test_split(path,neg_path=None, split = 0.2):
    print("------ PROCESS STARTED -------")


    files = list(set([name[:-4] for name in os.listdir(path)])) ## removing duplicate names i.e. counting only number of images


    print (f"--- This folder has a total number of {len(files)} images---")
    random.seed(42)
    random.shuffle(files)

    test_size = int(len(files) * split)
    train_size = len(files) - test_size

    ## creating required directories

    os.makedirs(train_path_img, exist_ok = True)
    os.makedirs(train_path_label, exist_ok = True)
    os.makedirs(val_path_img, exist_ok = True)
    os.makedirs(val_path_label, exist_ok = True)


    ### ----------- copying images to train folder
    for filex in tqdm(files[:train_size]):
      if filex == 'classes':
          continue
      shutil.copy2(path + filex + '.png',f"{train_path_img}/" + filex + '.png' )
      shutil.copy2(path + filex + '.txt', f"{train_path_label}/" + filex + '.txt')



    print(f"------ Training data created with 80% split {len(files[:train_size])} images -------")

    if neg_path:
        neg_images = list(set([name[:-4] for name in os.listdir(neg_path)])) ## removing duplicate names i.e. counting only number of images
        for filex in tqdm(neg_images):
            shutil.copy2(neg_path+filex+ ".png", f"{train_path_img}/" + filex + '.png')

        print(f"------ Total  {len(neg_images)} negative images added to the training data -------")

        print(f"------ TOTAL Training data created with {len(files[:train_size]) + len(neg_images)} images -------")



    ### copytin images to validation folder
    for filex in tqdm(files[train_size:]):
      if filex == 'classes':
          continue
      # print("running")
      shutil.copy2(path + filex + '.png', f"{val_path_img}/" + filex + '.png' )
      shutil.copy2(path + filex + '.txt', f"{val_path_label}/" + filex + '.txt')

    print(f"------ Testing data created with a total of {len(files[train_size:])} images ----------")

    print("------ TASK COMPLETED -------")

## spliting the data into train-test and creating train.txt and test.txt files
# train_test_split('/content/drive/MyDrive/custom_notebooks/yolo_data/')

### for label_tag
train_test_split('/content/drive/MyDrive/VISKOM/data/') ### without negative images
# train_test_split('./data/','./negative_images/') ### if you want to feed negative images

------ PROCESS STARTED -------
--- This folder has a total number of 120 images---


  0%|          | 0/96 [00:00<?, ?it/s]

------ Training data created with 80% split 96 images -------


  0%|          | 0/24 [00:00<?, ?it/s]

------ Testing data created with a total of 24 images ----------
------ TASK COMPLETED -------


In [ ]:
import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.0.237 🚀 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 26.3/78.2 GB disk)


## Training

In [ ]:
!yolo task=detect mode=train model=yolov8s.pt data=/content/drive/MyDrive/VISKOM/dataset.yaml epochs=100 imgsz=640 batch=8 project=/content/drive/MyDrive/VISKOM/training_results name=GuntingBatuKertas

Ultralytics YOLOv8.0.237 🚀 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8s.pt, data=/content/drive/MyDrive/VISKOM/dataset.yaml, epochs=100, time=None, patience=50, batch=8, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=/content/drive/MyDrive/VISKOM/training_results, name=GuntingBatuKertas4, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, sa

## Inferencing

In [ ]:
!yolo task=detect mode=predict model=/content/drive/MyDrive/VISKOM/training_results/GuntingBatuKertas/weights/best.pt conf=0.55 source=/content/drive/MyDrive/VISKOM/test

Ultralytics YOLOv8.0.238 🚀 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 11126745 parameters, 0 gradients, 28.4 GFLOPs

image 1/90 /content/drive/MyDrive/VISKOM/test/000b2478-e5dFezMV6TxIvNXA.png: 448x640 1 Batu, 65.4ms
image 2/90 /content/drive/MyDrive/VISKOM/test/034f8f6f-fnV8zv0atHjQzfhV.png: 448x640 1 Batu, 12.8ms
image 3/90 /content/drive/MyDrive/VISKOM/test/058072b1-xqCLaIfsVBFNf63M.png: 448x640 1 Gunting, 12.8ms
image 4/90 /content/drive/MyDrive/VISKOM/test/0a6bf70f-7gcPqW9meL8zJHFF.png: 448x640 1 Gunting, 12.8ms
image 5/90 /content/drive/MyDrive/VISKOM/test/0ed198ec-WQlZZHmfKvMFfUE8.png: 448x640 1 Batu, 12.8ms
image 6/90 /content/drive/MyDrive/VISKOM/test/12197475-6HFAfRozG6L6n9fl.png: 448x640 1 Gunting, 12.7ms
image 7/90 /content/drive/MyDrive/VISKOM/test/12484322-1Tt9U9SzrOm98Xtw.png: 448x640 1 Kertas, 12.8ms
image 8/90 /content/drive/MyDrive/VISKOM/test/12a3ceb3-5Zca8AdXnpDvoz18.png: 448x640 1 Gunting, 12.7ms
image 9/90 /cont

In [ ]:
!cp -r /content/runs/detect/predict /content/drive/MyDrive/VISKOM/output